In [1]:
import FirstCycling as FC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [2]:
fc = FC.FirstCycling()

In [3]:
fc.get_calendar(year=2021, month=3, cat='WT')

No race for this category in this month


In [18]:
teams =fc.get_teams(year=[2022,2021], cat=['WWT'])

In [19]:
display(teams)

,team,country,bike,wt_wins,uci_wins,uci_ranking,gender,year,category,team_url,team_id
0,CANYON//SRAM Racing,Germany,Canyon,0,0,3,W,2022,WWT,https://firstcycling.com/team.php?l=23744,23744
1,EF Education-TIBCO-SVB,USA,Cannondale,0,0,22,W,2022,WWT,https://firstcycling.com/team.php?l=27926,27926
2,FDJ-Nouvelle Aquitaine-Futuroscope,France,Lapierre,0,1,6,W,2022,WWT,https://firstcycling.com/team.php?l=21872,21872
3,Human Powered Health,USA,Felt,0,1,15,W,2022,WWT,https://firstcycling.com/team.php?l=28260,28260
4,Liv Racing Xstra,Netherlands,Liv,0,1,17,W,2022,WWT,https://firstcycling.com/team.php?l=27874,27874
5,Movistar Team,Spain,Canyon,0,4,7,W,2022,WWT,https://firstcycling.com/team.php?l=22296,22296
6,Roland Cogeas Edelweiss Squad,Switzerland,Look,0,0,31,W,2022,WWT,https://firstcycling.com/team.php?l=28433,28433
7,Team BikeExchange-Jayco,Australia,Liv,0,1,10,W,2022,WWT,https://firstcycling.com/team.php?l=22809,22809
8,Team DSM,Netherlands,Scott,1,3,5,W,2022,WWT,https://firstcycling.com/team.php?l=21877,21877
9,Team Jumbo-Visma,Netherlands,Cervélo,0,0,9,W,2022,WWT,https://firstcycling.com/team.php?l=27895,27895


In [20]:
teams_dict_with_riders = {}
for index in range(len(teams['team_id'])):
    team = teams['team'].loc[index] + " " + teams['year'].loc[index]
    full_url = teams['team_url'].loc[index]
    teams_dict_with_riders[team] = fc.get_riders(full_url)


display(teams_dict_with_riders)

{'CANYON//SRAM Racing 2022':                      Name Age    Nationality Race Days Race Km      ID
 0       Amialiusik, Alena  32        Belarus         6     761   89534
 1           Barnes, Alice  26  Great Britain         6     669   89637
 2          Bossuyt, Shari  21        Belgium         7     795   96695
 3          Bradbury, Neve  19      Australia         5     588  113734
 4          Chabbey, Elise  28    Switzerland         7     917   92095
 5       Cromwell, Tiffany  33      Australia         1     136   89538
 6           Dygert, Chloé  25            USA         1     128   89795
 7            Harris, Ella  23    New Zealand         6     709   91960
 8         Harvey, Mikayla  23    New Zealand         1     141   89819
 9             Klein, Lisa  25        Germany         6     696   89551
 10  Niewiadoma, Katarzyna  27         Poland         6     748   89419
 11          Oudeman, Maud  18    Netherlands       286          156233
 12        Paladin, Soraya  28      

In [22]:
team_id =23744

team_url = "https://firstcycling.com/team.php?l="+str(team_id)+"&riders=2"
soup = BeautifulSoup(requests.get(team_url).text,'lxml')
tables = soup.find_all('table', class_='sortTabell')
print(tables)
for table in tables:
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    names = []
    age =[]
    nationalities=[]
    racedays =[]
    racekm=[]
    riders_id = []

    #get the riders' ID from the html file parsing
    riders_href =list(soup.select('a[href^="rider.php"]'))
    tags =[]
    for tag in riders_href:
        tags.append( str(tag) )
    for tag in tags:
        start = tag.find('rider.php?r=')+len('rider.php?r=')
        end = tag.find('&amp')
        riders_id.append(tag[start:end])

    #get riders info from the html
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        names.append(cols[0])
        age.append(cols[2])
        nationalities.append(cols[3])
        racedays.append(cols[8])
        racekm.append(cols[9])
df_riders = pd.DataFrame(list(zip(names, age, nationalities,racedays,racekm, riders_id)),
    columns =['Name', 'Age', 'Nationality', 'Race Days', 'Race Km','ID'])
df_riders

[<table class="sortTabell tablesorter">
<thead>
<tr>
<th colspan="5"></th>
<th colspan="3" style="text-align:center;">UCI Ranking</th>
<th colspan="4"></th>
</tr>
<tr>
<th width="25%">Name</th>
<th width="5%"></th>
<th style="text-align:center;" width="5%">Age</th>
<th>Nation</th>
<th align="center">No.</th>
<th align="center" colspan="2">Points</th>
<th style="text-align:center;" width="5%">Wins</th>
<th style="text-align:center;white-space:nowrap;" width="5%">Race days</th>
<th style="text-align:center;" width="5%">Km</th>
<th>Strengths</th>
</tr>
</thead>
<tbody>
<tr>
<td>
<a href="rider.php?r=89534&amp;y=2022" title="Alena Amialiusik">
					Amialiusik, <span class="small">Alena</span>
</a>
</td>
<td align="center" style="white-space:nowrap;"></td>
<td align="center">32</td><td style="white-space:nowrap;font-size:0.95em;">
<img align="center" class="flag" src="img/flag/BLR.png" title=""/>
<a href="nation.php?nat=BLR&amp;k=ri">Belarus</a>
</td>
<td align="center" style="border-left:1

,Name,Age,Nationality,Race Days,Race Km,ID
0,"Amialiusik, Alena",32,Belarus,6,761,89534
1,"Barnes, Alice",26,Great Britain,6,669,89637
2,"Bossuyt, Shari",21,Belgium,7,795,96695
3,"Bradbury, Neve",19,Australia,5,588,113734
4,"Chabbey, Elise",28,Switzerland,7,917,92095
5,"Cromwell, Tiffany",33,Australia,1,136,89538
6,"Dygert, Chloé",25,USA,1,128,89795
7,"Harris, Ella",23,New Zealand,6,709,91960
8,"Harvey, Mikayla",23,New Zealand,1,141,89819
9,"Klein, Lisa",25,Germany,6,696,89551
